# ADR (Anomaly Detection by workflow Relations)

ADR mines numerical relations from log data and uses the relations for anomaly detection.

In the following parts, we use the BGL logs as example to show the capability of ADR.

## load datasets

For ease of presentation, the raw BGL logs are already parsed into structured log events by Drain <sup>[1]</sup> and the event-count-matrices are evaluated and saved in "_data.zip_". Please unzip "_data.zip_" to ADR folder before running the demo code.

In [1]:
import numpy as np

log_paths = {'hdfs': 'data/Drain_result/hdfs/x_y_xColumns.npz',
             'bgl': 'data/Drain_result/bgl/x_y_xColumns.npz',
             'hd': 'data/Drain_result/Hadoop/x_y_xColumns.npz',
             'spirit':'data/Drain_result/spirit/x_y_xColumns.npz'}

log_datasets = {}
for name, log_path in log_paths.items():
    log_datasets[name] = np.load(log_path, allow_pickle=True)

## sADR (supervised, need labelled logs for training)

In [2]:
from ADR import preprocess
from ADR import sADR

train_numbers = [100, 150, 200, 250, 300, 350, 400, 450, 500]

for log_name, x_y_xColumns in log_datasets.items():
    print(f'====={log_name}=====')
    x, y, xColumns = x_y_xColumns['x'], x_y_xColumns['y'], x_y_xColumns['xColumns']

    for i in range(len(train_numbers)):
        train_number = train_numbers[i]
        print(f'-----train number:{train_number}-----')
        if i == 0:
            x_train, y_train, x_test, y_test = x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_num(x, y, num_train=train_number)
        else:
            x_train_adding, y_train_adding, x_test, y_test = preprocess.split_to_train_test_by_num(x, y, num_train=train_numbers[i]-train_numbers[i-1])
            x_train = np.concatenate((x_train, x_train_adding), axis=0)
            y_train = np.concatenate((y_train, y_train_adding), axis=0)

        model = sADR.sADR()
        model.fit(x_train, y_train)
        precision, recall, f1 = model.evaluate(x_train, y_train)
        print('Accuracy on training set:')
        print(f"precision, recall, f1: {[precision, recall, f1]}")

        precision, recall, f1 = model.evaluate(x_test, y_test)
        print('Accuracy on testing set:')
        print(f"precision, recall, f1: {[precision, recall, f1]}")

=====hdfs=====
-----train number:100-----
Accuracy on training set:
precision, recall, f1: [1.0, 1.0, 1.0]
Accuracy on testing set:
precision, recall, f1: [0.8933, 1.0, 0.9437]
-----train number:150-----
Accuracy on training set:
precision, recall, f1: [1.0, 1.0, 1.0]
Accuracy on testing set:
precision, recall, f1: [0.8933, 1.0, 0.9437]
-----train number:200-----
Accuracy on training set:
precision, recall, f1: [1.0, 1.0, 1.0]
Accuracy on testing set:
precision, recall, f1: [0.8933, 1.0, 0.9437]
-----train number:250-----
Accuracy on training set:
precision, recall, f1: [1.0, 1.0, 1.0]
Accuracy on testing set:
precision, recall, f1: [0.8934, 1.0, 0.9437]
-----train number:300-----
Accuracy on training set:
precision, recall, f1: [1.0, 1.0, 1.0]
Accuracy on testing set:
precision, recall, f1: [0.8933, 1.0, 0.9437]
-----train number:350-----
Accuracy on training set:
precision, recall, f1: [1.0, 1.0, 1.0]
Accuracy on testing set:
precision, recall, f1: [0.8934, 1.0, 0.9437]
-----train nu

## uADR (unsupervised, do not need labelled logs for training)

In [2]:
from ADR import preprocess

u_log_datasets_train_test = {}

u_train_ratios = {'hdfs': 0.5,
                'bgl': 0.5,
                'hd': 0.5,
                'spirit': 0.5}
for name, x_y_xColumns in log_datasets.items():
    if name in ['hdfs', 'bgl', 'hd', 'spirit']:
        print("========")
        print(name)
        x, y, xColumns = x_y_xColumns['x'], x_y_xColumns['y'], x_y_xColumns['xColumns']
        print(f'x shape: {x.shape}')
        x_train, y_train, x_test, y_test = preprocess.split_to_train_test_by_ratio(x, y, train_ratio=u_train_ratios[name])
        u_log_datasets_train_test[name] = [x_train, y_train, x_test, y_test]
        print(f'x_train shape:{x_train.shape}')
        print(f'x_test shape:{x_test.shape}')

hdfs
x shape: (575061, 48)
x_train shape:(287530, 48)
x_test shape:(287531, 48)
bgl
x shape: (69252, 384)
x_train shape:(34626, 384)
x_test shape:(34626, 384)
hd
x shape: (55, 347)
x_train shape:(27, 347)
x_test shape:(28, 347)
spirit
x shape: (517, 988)
x_train shape:(258, 988)
x_test shape:(259, 988)


In [3]:
from ADR import uADR

estimated_pN = 0.9

for log_name in u_log_datasets_train_test:
    print(f'====={log_name}=====')
    x_train, y_train, x_test, y_test = u_log_datasets_train_test[log_name]

    model = uADR.uADR(AN_ratio=1-estimated_pN, nrows_per_sample=10, nrounds=100)
    model.fit(x_train)
    precision, recall, f1 = model.evaluate(x_train, y_train)
    print('Accuracy on training set:')
    print(f"precision, recall, f1: {[precision, recall, f1]}")

    precision, recall, f1 = model.evaluate(x_test, y_test)
    print('Accuracy on testing set:')
    print(f"precision, recall, f1: {[precision, recall, f1]}")

=====hdfs=====
Accuracy on training set:
precision, recall, f1: [0.8941, 1.0, 0.9441]
Accuracy on testing set:
precision, recall, f1: [0.8926, 1.0, 0.9433]
=====bgl=====
Accuracy on training set:
precision, recall, f1: [0.7088, 0.5568, 0.6237]
Accuracy on testing set:
precision, recall, f1: [0.7152, 0.5648, 0.6311]
=====hd=====


/home/bolz/miniconda3/envs/python373/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy on training set:
precision, recall, f1: [0.0, 0.0, 0.0]
Accuracy on testing set:
precision, recall, f1: [0.75, 1.0, 0.8571]
=====spirit=====
Accuracy on training set:
precision, recall, f1: [0.2431, 1.0, 0.3912]
Accuracy on testing set:
precision, recall, f1: [0.2046, 1.0, 0.3397]


## References

[1] P. He, J. Zhu, Z. Zheng, and M. R. Lyu, “Drain: An Online Log Parsing Approach with Fixed Depth Tree,” in 2017 IEEE International Conference on Web Services (ICWS), Jun. 2017, pp. 33–40, doi: 10.1109/ICWS.2017.13.